In [1]:
import cv2
import numpy as np
from numpy import array
import matplotlib.image as mpimg


img = cv2.imread('greyscale_lenna.jpg')
gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

ar = array(gray_img)
print('Shape of lena image is: ', ar.shape)

kernel = np.array([[1, 2, 1],[2, 4, 2],[1, 2, 1]])/16
print('Shape of kernel is: ', kernel.shape)


Shape of lena image is:  (512, 512)
Shape of kernel is:  (3, 3)


In [2]:
#Calculating final output size

#Number of rows and columns in image matrix
i_row_num, i_col_num = ar.shape

#Number of rows and columns in kernel matrix
k_row_num, k_col_num = kernel.shape

#Calculating dimensions of output matrix
output_row_num = i_row_num + k_row_num - 1
output_col_num = i_col_num + k_col_num - 1
print('Dimension of output signal is: ', output_row_num, output_col_num)

Dimension of output signal is:  514 514


In [3]:
#zero-padding of filter matrix

k_zero_padded = np.pad(kernel, ((output_row_num - k_row_num, 0),
          (0, output_col_num - k_col_num)),
      'constant', constant_values = 0)
print('Zero-padded filter matrix: ', k_zero_padded)

Zero-padded filter matrix:  [[0.     0.     0.     ... 0.     0.     0.    ]
 [0.     0.     0.     ... 0.     0.     0.    ]
 [0.     0.     0.     ... 0.     0.     0.    ]
 ...
 [0.0625 0.125  0.0625 ... 0.     0.     0.    ]
 [0.125  0.25   0.125  ... 0.     0.     0.    ]
 [0.0625 0.125  0.0625 ... 0.     0.     0.    ]]


In [4]:
#toeplitz matrix for each row of the zero padded filter

from scipy.linalg import toeplitz

toeplitz_list = []

for i in range(k_zero_padded.shape[0] - 1, -1, -1):
    c = k_zero_padded[i, : ]
    r = np.r_[c[0], np.zeros(i_col_num - 1)]
    toeplitz_m = toeplitz(c, r)
    toeplitz_list.append(toeplitz_m)
    #print('Kernel '+ str(i)+'\n', toeplitz_m)

In [5]:
# doubly blocked toeplitz indices

c = range(1, k_zero_padded.shape[0] + 1)
r = np.r_[c[0], np.zeros(i_row_num - 1, dtype = int)]
doubly_indices = toeplitz(c, r)
print('Doubly indices \n', doubly_indices)

Doubly indices 
 [[  1   0   0 ...   0   0   0]
 [  2   1   0 ...   0   0   0]
 [  3   2   1 ...   0   0   0]
 ...
 [512 511 510 ...   3   2   1]
 [513 512 511 ...   4   3   2]
 [514 513 512 ...   5   4   3]]


In [6]:
# creat doubly blocked matrix with zero values


toeplitz_shape = toeplitz_list[0].shape
h = toeplitz_shape[0] * doubly_indices.shape[0]
w = toeplitz_shape[1] * doubly_indices.shape[1]
doubly_blocked_shape = [h, w]
doubly_blocked = np.zeros(doubly_blocked_shape, dtype='uint8')

b_h, b_w = toeplitz_shape
for i in range(doubly_indices.shape[0]):
    for j in range(doubly_indices.shape[1]):
        start_i = i * b_h
        start_j = j * b_w
        end_i = start_i + b_h
        end_j = start_j + b_w
        doubly_blocked[start_i : end_i, start_j : end_j] = toeplitz_list[doubly_indices[i, j] - 1]
        
#print('Doubly_blocked', doubly_blocked)

MemoryError: Unable to allocate 64.5 GiB for an array with shape (264196, 262144) and data type uint8

In [7]:
#Converting input matrix to a vector

def matrix_to_vector(inp):
    input_h, input_w = inp.shape
    output_vector = np.zeros(input_h*input_w, dtype=inp.dtype)
    # flip the input matrix up-down because last row should go first
    inp = np.flipud(inp) 
    for i,row in enumerate(inp):
        st = i*input_w
        nd = st + input_w
        output_vector[st:nd] = row
        
    return output_vector

In [9]:
# call the function

vectorized_i = matrix_to_vector(ar)
print('vectorized_input: ', vectorized_i)

vectorized_input:  [ 34  34  41 ... 144 132 101]


In [12]:
# get result of the convolution by matrix mupltiplication

result_vector = np.matmul(doubly_blocked, vectorized_i)
print('result_vector: ', result_vector)

NameError: name 'doubly_blocked' is not defined

In [13]:
#reshape the result to a matrix form

def vector_to_matrix(inp, output_shape):
    output_h, output_w = output_shape
    output = np.zeros(output_shape, dtype=inp.dtype)
    for i in range(output_h):
        st = i*output_w
        nd = st + output_w
        output[i, :] = inp[st:nd]
    # flip the output matrix up-down to get correct result
    output=np.flipud(output)
    return output

In [14]:
# reshape the raw rsult to desired matrix form
out_shape = [output_row_num, output_col_num]
my_output = vector_to_matrix(result_vector, out_shape)

print('Result of implemented method: \n', my_output)

NameError: name 'result_vector' is not defined